## Install Library

In [1]:
!pip install pdf2image
!apt-get install poppler-utils
!pip install streamlit
!pip install pyngrok
!pip install numpy
!pip install "paddleocr==2.7.0"
!pip install paddlepaddle
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (271 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123599 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from pdf2image import convert_from_path
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from paddleocr import PaddleOCR, draw_ocr
import streamlit as st




In [7]:
%%writefile app.py
import streamlit as st
from paddleocr import PaddleOCR
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
import pandas as pd
ocr = PaddleOCR(lang='fr')

def detect_text(image):
    """
    Fonction pour détecter le texte dans l'image et dessiner des boîtes autour des mots détectés.
    Retourne également le texte détecté.
    """

    image_np = np.array(image)
    image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)


    output = ocr.ocr(image_cv)[0]


    boxes = [line[0] for line in output]
    texts = [line[1][0] for line in output]
    probabilities = [line[1][1] for line in output]


    for box, text in zip(boxes, texts):
        cv2.rectangle(image_cv, (int(box[0][0]), int(box[0][1])), (int(box[2][0]), int(box[2][1])), (0, 0, 255), 2)
        cv2.putText(image_cv, text, (int(box[0][0]), int(box[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (222, 0, 0), 2)

    return image_cv, boxes, texts, probabilities

def detect_horizontal_vertical(image, boxes):
    """
    Fonction pour détecter les zones horizontales et verticales dans une image.
    Retourne l'image avec les boîtes horizontales et verticales dessinées.
    """

    image_np = np.array(image)
    image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    image_height = image_cv.shape[0]
    image_width = image_cv.shape[1]

    horiz_boxes = []
    vert_boxes = []

    for box in boxes:
        x_h, x_v = 0, int(box[0][0])
        y_h, y_v = int(box[0][1]), 0
        width_h, width_v = image_width, int(box[2][0] - box[0][0])
        height_h, height_v = int(box[2][1] - box[0][1]), image_height

        horiz_boxes.append([x_h, y_h, x_h + width_h, y_h + height_h])
        vert_boxes.append([x_v, y_v, x_v + width_v, y_v + height_v])


        cv2.rectangle(image_cv, (x_h, y_h), (x_h + width_h, y_h + height_h), (0, 0, 255), 1)
        cv2.rectangle(image_cv, (x_v, y_v), (x_v + width_v, y_v + height_v), (0, 255, 0), 1)

    return image_cv, horiz_boxes, vert_boxes

def apply_non_max_suppression(horiz_boxes, vert_boxes, probabilities, image):
    """
    Appliquer la suppression non maximale aux boîtes horizontales et verticales.
    Retourne l'image avec les boîtes de suppression non maximale dessinées.
    """

    image_np = np.array(image)
    image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)


    horiz_out = tf.image.non_max_suppression(
        horiz_boxes,
        probabilities,
        max_output_size=1000,
        iou_threshold=0.1,
        score_threshold=float('-inf')
    )
    horiz_lines = np.sort(np.array(horiz_out))

    for val in horiz_lines:
        cv2.rectangle(image_cv, (int(horiz_boxes[val][0]), int(horiz_boxes[val][1])),
                      (int(horiz_boxes[val][2]), int(horiz_boxes[val][3])), (0, 0, 255), 1)


    vert_out = tf.image.non_max_suppression(
        vert_boxes,
        probabilities,
        max_output_size=1000,
        iou_threshold=0.1,
        score_threshold=float('-inf')
    )
    vert_lines = np.sort(np.array(vert_out))

    for val in vert_lines:
        cv2.rectangle(image_cv, (int(vert_boxes[val][0]), int(vert_boxes[val][1])),
                      (int(vert_boxes[val][2]), int(vert_boxes[val][3])), (255, 0, 0), 1)

    return image_cv, horiz_lines, vert_lines

def intersection(box_1, box_2):
    """
    Calculer l'intersection entre deux boîtes.
    """
    return [box_2[0], box_1[1], box_2[2], box_1[3]]

def iou(box_1, box_2):
    """
    Calculer l'Intersection over Union (IoU) entre deux boîtes.
    """
    x_1 = max(box_1[0], box_2[0])
    y_1 = max(box_1[1], box_2[1])
    x_2 = min(box_1[2], box_2[2])
    y_2 = min(box_1[3], box_2[3])

    inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
    if inter == 0:
        return 0

    box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
    box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

    return inter / float(box_1_area + box_2_area - inter)

def generate_csv(horiz_lines, vert_lines, horiz_boxes, vert_boxes, boxes, texts):
    """
    Générer un fichier CSV avec la position des mots détectés.
    """
    unordered_boxes = []
    for i in vert_lines:
        unordered_boxes.append(vert_boxes[i][0])
    ordered_boxes = np.argsort(unordered_boxes)

    out_array = [["" for _ in range(len(vert_lines))] for _ in range(len(horiz_lines))]

    for i in range(len(horiz_lines)):
        for j in range(len(vert_lines)):
            resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]])

            for b in range(len(boxes)):
                the_box = [boxes[b][0][0], boxes[b][0][1], boxes[b][2][0], boxes[b][2][1]]
                if iou(resultant, the_box) > 0.1:
                    out_array[i][j] = texts[b]

    df = pd.DataFrame(out_array)
    return df



st.image("logo.png", width=200)
st.title("Détection de texte OCR avec suppression non maximale")


uploaded_file = st.file_uploader("Téléchargez une image", type=['png', 'jpg', 'jpeg'])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Image téléchargée', use_column_width=True)


    result_image, boxes, extracted_texts, probabilities = detect_text(image)


    result_image_pil = Image.fromarray(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    st.image(result_image_pil, caption='Image avec détection de texte', use_column_width=True)


    st.write("Texte détecté dans l'image :")
    for text in extracted_texts:
        st.write(text)


    horiz_vert_image, horiz_boxes, vert_boxes = detect_horizontal_vertical(image, boxes)

    horiz_vert_image_pil = Image.fromarray(cv2.cvtColor(horiz_vert_image, cv2.COLOR_BGR2RGB))
    st.image(horiz_vert_image_pil, caption='Image avec boîtes horizontales et verticales', use_column_width=True)


    suppressed_image, horiz_lines, vert_lines = apply_non_max_suppression(horiz_boxes, vert_boxes, probabilities, image)


    suppressed_image_pil = Image.fromarray(cv2.cvtColor(suppressed_image, cv2.COLOR_BGR2RGB))
    st.image(suppressed_image_pil, caption='Image avec suppression non maximale', use_column_width=True)


    df = generate_csv(horiz_lines, vert_lines, horiz_boxes, vert_boxes, boxes, extracted_texts)

    st.write("Résultats OCR sous forme de tableau :")
    st.dataframe(df)

    csv = df.to_csv(index=False).encode('utf-8')

    st.download_button(
        label="Télécharger les résultats sous forme de CSV",
        data=csv,
        file_name='resultats_ocr.csv',
        mime='text/csv',
    )


Overwriting app.py


## Script NGROK

In [3]:
import os
from pyngrok import ngrok
ngrok.set_auth_token('2l3ZMJuiCVuO8fONnKFWe1uXW9x_3VdvGuu9JchHseJ7oL8zi')

In [5]:
!streamlit run  app.py &>/dev/null &
from pyngrok import ngrok
public_url = ngrok.connect(addr="8501", proto="http")
print(f'Application publique URL: {public_url}')


Application publique URL: NgrokTunnel: "https://af93-34-139-130-118.ngrok-free.app" -> "http://localhost:8501"
